In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import polars as pl
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mooccubex-relations/course-teacher.txt
/kaggle/input/mooccubex-relations/concept-comment.txt
/kaggle/input/mooccubex-relations/user-video.json
/kaggle/input/mooccubex-relations/course-comment.txt
/kaggle/input/mooccubex-relations/concept-problem.txt
/kaggle/input/mooccubex-relations/user-reply.txt
/kaggle/input/mooccubex-relations/concept-video.txt
/kaggle/input/mooccubex-relations/course-school.txt
/kaggle/input/mooccubex-relations/user-comment.txt
/kaggle/input/mooccubex-relations/exercise-problem.txt
/kaggle/input/mooccubex-relations/concept-paper.txt
/kaggle/input/mooccubex-relations/reply-reply.txt
/kaggle/input/mooccubex-relations/course-field.json
/kaggle/input/mooccubex-relations/video_id-ccid.txt
/kaggle/input/mooccubex-relations/user-xiaomu.json
/kaggle/input/mooccubex-relations/user-problem.json
/kaggle/input/mooccubex-relations/comment-reply.txt
/kaggle/input/mooccubex-relations/concept-other.txt
/kaggle/input/preprocessing-course-json/course.json
/kaggle/inpu

In [2]:
course = pd.read_json('/kaggle/input/preprocessing-course-json/course.json',lines=True)

In [3]:
def convert_dataframe(df):
    """Chuyển đổi giữa Pandas DataFrame và Polars DataFrame"""
    if isinstance(df, pd.DataFrame):
        return pl.from_pandas(df)  # Chuyển từ Pandas sang Polars
    elif isinstance(df, pl.DataFrame):
        return df.to_pandas()  # Chuyển từ Polars sang Pandas
    else:
        raise TypeError("Input phải là Pandas DataFrame hoặc Polars DataFrame!")

In [4]:
course

,id,name,field,prerequisites,about,resource,about_trans,prerequisites_trans,name_trans
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",,通过老师导读，同学们可深入这一经典文本内部，得以纵览千年历史，提升国学素养，体味人生智慧。,"[{'titles': ['第一课 导论与三家分晋', '导论', '导论'], 'reso...","through the teacher's guidance, students can g...",,"introduction to ""zizhi tongjian"""
1,C_584329,微积分——极限理论与一元函数,"[应用经济学, 数学, 物理学, 理论经济学]",,本课程是理工科的一门数学基础课，系统、全面地介绍了一元函数微积分学。课程既保持了数学的严谨和...,"[{'titles': ['序言', '序言', '序言'], 'resource_id':...",this course is a basic mathematics course in s...,,calculus——extreme theory and univariate function
2,C_584381,新闻摄影,"[艺术学, 新闻传播学]",,掌握基本的摄影技能，了解图片新闻的工作方式，训练对生活的观察和热爱，发展对图像的审美和批评能...,"[{'titles': ['第一章 绪论', '第一讲 引言1', '引言1'], 'res...","master basic photography skills, understand th...",,news photography
3,C_597208,数据挖掘：理论与算法,[计算机科学与技术],,最有趣的理论+最有用的算法=不得不学的数据科学。,"[{'titles': ['走进数据科学：博大精深，美不胜收', '整装待发', 'Vide...",the most interesting theory + the most useful ...,,data mining: theory and algorithm
4,C_597225,大学计算机,[],,大学计算机课程将以计算思维为导向，以计算机原理、概念为基础，以新技术新方法为牵引，以创新思维...,"[{'titles': ['第1周： 基于计算机的问题求解', '课程介绍', '开篇'],...",the university computer courses will be guided...,,university computer
...,...,...,...,...,...,...,...,...,...
3776,C_2338076,（疾风计划）软件工程,[],,疾风计划2021【首期限额招募】进行中\n名校名师丨专属助教丨专业认证丨系统全面丨修炼内功丨...,"[{'titles': ['第1章 初识软件工程 ', '1.1 软件无处不在 ', '讲课...",breeze plan 2021 [first term recruitment] is i...,,(breathing plan) software engineering
3777,C_2341259,（疾风计划）面向对象程序设计（C++）,[],,疾风计划2021【首期限额招募】进行中\n名校名师丨专属助教丨专业认证丨系统全面丨修炼内功丨...,"[{'titles': ['第一讲 课程简介与编程环境', '1.0 课程定位、教学内容',...",breeze plan 2021 [first term recruitment] is i...,,(breathing project) object-oriented programmin...
3778,C_2337996,（疾风计划）数据结构(下),[],,疾风计划2021【首期限额招募】进行中\n名校名师丨专属助教丨专业认证丨系统全面丨修炼内功丨...,"[{'titles': ['第零章 ', '选课之前', '宣传片'], 'resourc...",breeze plan 2021 [first term recruitment] is i...,,(breath plan) data structure (part 2)
3779,C_1945689,机器学习训练营,[],,清华张敏老师带你12周掌握机器学习！8大经典算法 +7大实训 + 直播点评 + 博士助教+云...,"[{'titles': ['序-开营仪式及学习课件', None, '1.1_初识机器学习'...",teacher zhang min from tsinghua university wil...,,machine learning bootcamp


In [5]:
pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from deep_translator import GoogleTranslator

def translate_list_column_pandas(df, col_name, source_lang="zh-TW", target_lang="en"):
    """Dịch cột chứa danh sách lĩnh vực từ tiếng Trung sang tiếng Anh trong Pandas DataFrame và lưu vào cột mới."""
    translator = GoogleTranslator(source=source_lang, target=target_lang)
    
    df[col_name + "_trans"] = df[col_name].apply(lambda x: [translator.translate(field) for field in x] if isinstance(x, list) else x)
    return df

In [7]:
# Ví dụ sử dụng với Pandas
print("Trước khi dịch:\n")      
display(course.head())
df_pandas_translated = translate_list_column_pandas(course, "field")
print("Sau khi dịch:\n") 
display(df_pandas_translated.head())

Trước khi dịch:



,id,name,field,prerequisites,about,resource,about_trans,prerequisites_trans,name_trans
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",,通过老师导读，同学们可深入这一经典文本内部，得以纵览千年历史，提升国学素养，体味人生智慧。,"[{'titles': ['第一课 导论与三家分晋', '导论', '导论'], 'reso...","through the teacher's guidance, students can g...",,"introduction to ""zizhi tongjian"""
1,C_584329,微积分——极限理论与一元函数,"[应用经济学, 数学, 物理学, 理论经济学]",,本课程是理工科的一门数学基础课，系统、全面地介绍了一元函数微积分学。课程既保持了数学的严谨和...,"[{'titles': ['序言', '序言', '序言'], 'resource_id':...",this course is a basic mathematics course in s...,,calculus——extreme theory and univariate function
2,C_584381,新闻摄影,"[艺术学, 新闻传播学]",,掌握基本的摄影技能，了解图片新闻的工作方式，训练对生活的观察和热爱，发展对图像的审美和批评能...,"[{'titles': ['第一章 绪论', '第一讲 引言1', '引言1'], 'res...","master basic photography skills, understand th...",,news photography
3,C_597208,数据挖掘：理论与算法,[计算机科学与技术],,最有趣的理论+最有用的算法=不得不学的数据科学。,"[{'titles': ['走进数据科学：博大精深，美不胜收', '整装待发', 'Vide...",the most interesting theory + the most useful ...,,data mining: theory and algorithm
4,C_597225,大学计算机,[],,大学计算机课程将以计算思维为导向，以计算机原理、概念为基础，以新技术新方法为牵引，以创新思维...,"[{'titles': ['第1周： 基于计算机的问题求解', '课程介绍', '开篇'],...",the university computer courses will be guided...,,university computer


Sau khi dịch:



,id,name,field,prerequisites,about,resource,about_trans,prerequisites_trans,name_trans,field_trans
0,C_584313,《资治通鉴》导读,"[历史学, 中国语言文学]",,通过老师导读，同学们可深入这一经典文本内部，得以纵览千年历史，提升国学素养，体味人生智慧。,"[{'titles': ['第一课 导论与三家分晋', '导论', '导论'], 'reso...","through the teacher's guidance, students can g...",,"introduction to ""zizhi tongjian""","[History, Chinese Language and Literature]"
1,C_584329,微积分——极限理论与一元函数,"[应用经济学, 数学, 物理学, 理论经济学]",,本课程是理工科的一门数学基础课，系统、全面地介绍了一元函数微积分学。课程既保持了数学的严谨和...,"[{'titles': ['序言', '序言', '序言'], 'resource_id':...",this course is a basic mathematics course in s...,,calculus——extreme theory and univariate function,"[Applied Economics, math, physics, Theoretical..."
2,C_584381,新闻摄影,"[艺术学, 新闻传播学]",,掌握基本的摄影技能，了解图片新闻的工作方式，训练对生活的观察和热爱，发展对图像的审美和批评能...,"[{'titles': ['第一章 绪论', '第一讲 引言1', '引言1'], 'res...","master basic photography skills, understand th...",,news photography,"[Art Studies, Journalism Communication]"
3,C_597208,数据挖掘：理论与算法,[计算机科学与技术],,最有趣的理论+最有用的算法=不得不学的数据科学。,"[{'titles': ['走进数据科学：博大精深，美不胜收', '整装待发', 'Vide...",the most interesting theory + the most useful ...,,data mining: theory and algorithm,[Computer Science and Technology]
4,C_597225,大学计算机,[],,大学计算机课程将以计算思维为导向，以计算机原理、概念为基础，以新技术新方法为牵引，以创新思维...,"[{'titles': ['第1周： 基于计算机的问题求解', '课程介绍', '开篇'],...",the university computer courses will be guided...,,university computer,[]


In [8]:
en_course=df_pandas_translated[["id","name_trans","field_trans","about_trans","prerequisites_trans"]]
en_course

,id,name_trans,field_trans,about_trans,prerequisites_trans
0,C_584313,"introduction to ""zizhi tongjian""","[History, Chinese Language and Literature]","through the teacher's guidance, students can g...",
1,C_584329,calculus——extreme theory and univariate function,"[Applied Economics, math, physics, Theoretical...",this course is a basic mathematics course in s...,
2,C_584381,news photography,"[Art Studies, Journalism Communication]","master basic photography skills, understand th...",
3,C_597208,data mining: theory and algorithm,[Computer Science and Technology],the most interesting theory + the most useful ...,
4,C_597225,university computer,[],the university computer courses will be guided...,
...,...,...,...,...,...
3776,C_2338076,(breathing plan) software engineering,[],breeze plan 2021 [first term recruitment] is i...,
3777,C_2341259,(breathing project) object-oriented programmin...,[],breeze plan 2021 [first term recruitment] is i...,
3778,C_2337996,(breath plan) data structure (part 2),[],breeze plan 2021 [first term recruitment] is i...,
3779,C_1945689,machine learning bootcamp,[],teacher zhang min from tsinghua university wil...,


In [9]:
course_field = en_course[["id","field_trans"]]

In [10]:
def explode_list_column(df, col_name):
    """Phân rã (explode) cột kiểu danh sách, hỗ trợ cả Pandas và Polars."""
    if isinstance(df, pd.DataFrame):
        return df.explode(col_name).reset_index(drop=True)  # Pandas explode
    
    elif isinstance(df, pl.DataFrame):
        return df.explode(col_name)  # Polars explode
    
    else:
        raise TypeError("Chỉ hỗ trợ Pandas hoặc Polars DataFrame")
def dropna_rows(df, col_name=None):
    """Xoá các dòng chứa NaN (None) trong một cột hoặc toàn bộ DataFrame."""
    if isinstance(df, pd.DataFrame):  # Pandas
        return df.dropna(subset=[col_name]) if col_name else df.dropna()
    
    elif isinstance(df, pl.DataFrame):  # Polars
        return df.drop_nulls(col_name) if col_name else df.drop_nulls()
    
    else:
        raise TypeError("Chỉ hỗ trợ Pandas hoặc Polars DataFrame")

In [11]:
course_field = explode_list_column(course_field,"field_trans")

In [12]:
def groupby_id(df, col_name):
    """Group By theo 'id' và gộp các giá trị thành danh sách, hỗ trợ Pandas & Polars."""
    if isinstance(df, pd.DataFrame):  # Pandas
        return df.groupby("id", as_index=False).agg({col_name: lambda x: list(x.dropna())})
    
    elif isinstance(df, pl.DataFrame):  # Polars
        return df.groupby("id").agg(pl.col(col_name).drop_nulls().list())
    
    else:
        raise TypeError("Chỉ hỗ trợ Pandas hoặc Polars DataFrame")

In [13]:
dropna_rows(course_field,"field_trans")

,id,field_trans
0,C_584313,History
1,C_584313,Chinese Language and Literature
2,C_584329,Applied Economics
3,C_584329,math
4,C_584329,physics
...,...,...
3697,C_697791,Computer Science and Technology
3698,C_682189,Computer Science and Technology
3700,C_735157,Computer Science and Technology
3701,C_677218,Computer Science and Technology


In [14]:
course = convert_dataframe(course)
course = explode_list_column(course,'resource')

In [15]:
r_course_field = pd.read_json('/kaggle/input/mooccubex-relations/course-field.json',lines=True)
r_course_field = translate_list_column_pandas(r_course_field,'field')


In [16]:
r_course_field['course_id'] = r_course_field['course_id'].apply(lambda x: f"C_{x}")

In [17]:
display(r_course_field.head(),r_course_field.describe(),en_course.head())

,course_id,course_name,field,field_trans
0,C_584313,《资治通鉴》导读,"[中国语言文学, 历史学]","[Chinese Language and Literature, History]"
1,C_681932,“做中学”Java程序设计,[计算机科学与技术],[Computer Science and Technology]
2,C_674962,《红楼梦》的空间艺术,[中国语言文学],[Chinese Language and Literature]
3,C_682709,《纯粹理性批判》导论,[哲学],[philosophy]
4,C_682635,《统万城》导读,[历史学],[History]


,course_id,course_name,field,field_trans
count,632,632,632,632
unique,632,610,172,172
top,C_584313,中国近现代史纲要,[计算机科学与技术],[Computer Science and Technology]
freq,1,4,64,64


,id,name_trans,field_trans,about_trans,prerequisites_trans
0,C_584313,"introduction to ""zizhi tongjian""","[History, Chinese Language and Literature]","through the teacher's guidance, students can g...",
1,C_584329,calculus——extreme theory and univariate function,"[Applied Economics, math, physics, Theoretical...",this course is a basic mathematics course in s...,
2,C_584381,news photography,"[Art Studies, Journalism Communication]","master basic photography skills, understand th...",
3,C_597208,data mining: theory and algorithm,[Computer Science and Technology],the most interesting theory + the most useful ...,
4,C_597225,university computer,[],the university computer courses will be guided...,


In [18]:
df = pd.concat([
    r_course_field[['course_id','field_trans']],
    en_course[['id','field_trans']].rename(columns={'id':'course_id'})
],
          axis=0)
df = explode_list_column(df,'field_trans').drop_duplicates().dropna()
display(df.head(),df.describe())

,course_id,field_trans
0,C_584313,Chinese Language and Literature
1,C_584313,History
2,C_681932,Computer Science and Technology
3,C_674962,Chinese Language and Literature
4,C_682709,philosophy


,course_id,field_trans
count,793,793
unique,632,82
top,C_696753,Computer Science and Technology
freq,7,74


In [19]:
df['course_id'].value_counts()

course_id
C_696753    7
C_681056    5
C_682313    4
C_597305    4
C_584329    4
           ..
C_680970    1
C_680817    1
C_681647    1
C_696664    1
C_681331    1
Name: count, Length: 632, dtype: int64

In [20]:
course_field = df.rename(columns={'field_trans':'course_field'})
course_field['course_field'] = course_field['course_field'].apply(lambda x: x.lower())
# course_field = course_field.sort_values(by=['course_field'])
course_field.head()
# .groupby('course_id').agg(list).reset_index()

,course_id,course_field
0,C_584313,chinese language and literature
1,C_584313,history
2,C_681932,computer science and technology
3,C_674962,chinese language and literature
4,C_682709,philosophy


In [21]:
course_field.to_csv('course_field.csv')